# Kubernetes

In [94]:
# WEBDAV at http://10.15.136.41:30080/

import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import Javascript, display
import uuid

style = {'description_width': '250px'}
layout = {'width': '500px'}

def addVisText(key='',value='',disabled=False, layout=layout):
    text = widgets.Text(description = key,value = value, style=style, layout=layout,disabled=disabled)
    display(text)  
    return text
    
def addVisCheckbox(key='',value=False):
    text = widgets.Checkbox(description = key,value = value, style=style, layout=layout)
    display(text)   
    return text

def getPort(peername='', name='peer-gossip'):
    callopt= 'jsonpath=\"{.spec.ports[?(@.name==\'' + name + '\')].nodePort}\"'
    try:    
        callProcess  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'get','service',peername,'-o', callopt])
        return str(callProcess)[3:-2]
    except Exception as e:
        return str(e)

def setPeerPorts(peername):
    os.environ['GOSSIP_PORT'] = getPort(peername=peername, name='peer-gossip')
    os.environ['PEER_LISTEN_PORT'] = getPort(peername=peername, name='peer-listen')
    os.environ['PEER_CHAINCODE_PORT'] = getPort(peername=peername, name='peer-chaincode')
    
def createConfig(peername='',peer_listen_port='',peer_gossip_port=''):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                          '--from-literal=hostname=kubernetes.research.dev.seeburger.de',
                                          '--from-literal=org_name=' + peername,
                                          '--from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:' + peer_listen_port,
                                          '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:' + peer_gossip_port,
                                          '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
        return str(output)
    except Exception as e:
        return str(e)  
    
#!/home/jovyan/work/usr/bin/kubectl create configmap hl-fabric-peer-$PEER_NAME \
#         --from-literal=hostname=kubernetes.research.dev.seeburger.de \
#         --from-literal=org_name=$PEER_NAME \
#         --from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:$PEER_LISTEN_PORT \
#         --from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:$GOSSIP_PORT \
#         --from-literal=CORE_PEER_LOCALMSPID=${PEER_NAME}MSP
            
def getPod(peername):
    try:    
        callProcess  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'get','pod','-l','app=' + peername,'-o', 
                                                'jsonpath=\"{.items[0].metadata.name}\"'])
        return str(callProcess)[3:-2]
    except Exception as e:
        return str(e)     
    
def getPods():
    !/home/jovyan/work/usr/bin/kubectl get pods
        
def setPeerEnvironment(peername):
    os.environ['POD_NAME'] = getPod(peername)
    os.environ['ORDERER_IP'] = '10.15.128.97'
    os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
    os.environ['CHANNEL_NAME'] = 'mychannel'  
    setPeerPorts(peername)
    
def executeKubectlCmd(cmd):
    try:    
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl'] + cmd)
        return output
    except Exception as e:
        return str(e)     
    
def deletePeer(peername):   
    output  = executeKubectlCmd(['delete','configmap','hl-fabric-peer-' + peername])
    output  = executeKubectlCmd(['delete','-f','/home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/' + peername + '/k8s-peer.yaml'])
    output  = executeKubectlCmd(['delete', 'service', peername])
    return output
    
#!/home/jovyan/work/usr/bin/kubectl get configmap -o json     
def getConfigmap(): 
    return json.loads(executeKubectlCmd(['get','configmap', '-o', 'json']).decode('ascii'))
   
    
#!/home/jovyan/work/usr/bin/kubectl  exec --stdin --tty $POD_NAME  -c scray-peer-cli /bin/bash "echo 'export ORDERER_CA=/tmp/tlsca.example.com-cert.pem' >> /root/.bashrc"    

#!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c scray-peer-cli -- /bin/bash -c "echo 'export ORDERER_CA=/tmp/tlsca.example.com-cert.pem' >> /root/.bashrc"

def executePeerCmd(pod_name,cmd):
    try:  
        cmd1 = "source /root/.bashrc && "
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec','-it', pod_name, '-c', 'scray-peer-cli','--', '/bin/bash','-c',cmd1 + cmd])
        return output.decode('ascii')
    except Exception as e:
        return str(e)

def addEnvVariable(pod_name, key,value):
    try:  
        cmd = "echo \'export '" + key + '=' + value + "'\' >> /root/.bashrc"
        #cmd = "echo 1"
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec','-it', pod_name, '-c', 'scray-peer-cli','--', '/bin/bash','-c',cmd])
        return output.decode('ascii')
    except Exception as e:
        return str(e)
       
def addBashEnvToPeer(peer):        
    pod_name    = peer.pod_name
    listen_port = peer.listen_port
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli touch /root/.bashrc
    addEnvVariable(pod_name, 'ORDERER_CA', '/tmp/tlsca.example.com-cert.pem')
    addEnvVariable(pod_name, 'ORDERER_HOSTNAME', 'orderer.example.com')
    addEnvVariable(pod_name, 'CHANNEL_NAME', 'mychannel')
    #addEnvVariable(pod_name, 'CORE_PEER_ADDRESS', pod_name + ':' + listen_port)
    addEnvVariable(pod_name, 'CORE_PEER_ADDRESS', 'peer0.kubernetes.research.dev.seeburger.de:' + listen_port)
    #addEnvVariable(pod_name, 'CORE_PEER_ADDRESS', 'kubernetes.research.dev.seeburger.de:' + listen_port)
    addEnvVariable(pod_name, 'CORE_PEER_MSPCONFIGPATH', '/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp')
    addEnvVariable(pod_name, 'SHARED_FS_HOST', '10.15.136.41:30080')
    addEnvVariable(pod_name, 'SHARED_FS_USER', 'scray')
    addEnvVariable(pod_name, 'SHARED_FS_PW', 'scray')        
  
#!/home/jovyan/work/usr/bin/kubectl exec -it tanja1-65b7889d68-ghxpj -c scray-peer-cli -- /bin/bash -c "curl --user scray:scray http://10.15.136.41:30080/upload/org3_update_in_envelope.pb -o /tmp/org3_update_in_envelope.pb"
def sign(pod_name):
    #executePeerCmd(pod_name,"curl --user $SHARED_FS_USER:$SHARED_FS_PW http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem > /tmp/tlsca.example.com-cert.pem"
    executePeerCmd(pod_name,"curl --user scray:scray http://10.15.136.41:30080/upload/org3_update_in_envelope.pb -o /tmp/org3_update_in_envelope.pb")
    executePeerCmd(pod_name,"peer channel signconfigtx -f /tmp/org3_update_in_envelope.pb")
    executePeerCmd(pod_name,"curl --user scray:scray -T /tmp/org3_update_in_envelope.pb http://10.15.136.41:30080/upload/org3_update_in_envelope.pb")
       
def joinHyperledger(pod_name,orderer_ip,orderer_hostname,channel_name):    
    #setPeerEnvironment(peername)    
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME    
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec','--stdin', '--tty', pod_name, '-c', 'scray-peer-cli','--', '/bin/sh','/mnt/conf/peer_join.sh',
                                           orderer_ip,orderer_hostname,channel_name])
        return output
    except Exception as e:
        return str(e)
    
def installAndAproveChaincode(peer,cc_host='10.14.128.38',orderer_host='10.15.128.97'):
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK
    executePeerCmd(peer.pod_name,'/mnt/conf/install_and_approve_cc.sh ' + cc_host + ' ' + orderer_host) 
    

In [ ]:
addBashEnvToPeer(peer)
peer.name

In [ ]:
def deleteAllPeers(config):
    for item in config['items']:
        try:
            deletePeer(item['data']['org_name'])
        except Exception as e:
            print(item)
config = getConfigmap()  

deleteAllPeers(config)
#deletePeer('bbc2')

In [ ]:
#deletePeer('tanja5')
config = getConfigmap()
len(config['items'])

In [ ]:
# testing, print org names
for item in config['items']:
    try:
        print(item['data']['org_name'])
    except Exception as e:
        print(item)

In [3]:
class kubernetespeer:
    def __init__(self, name='',pod_name='',listen_port='',gossip_port=''):
        self.name = name
        self.pod_name = pod_name
        self.listen_port = listen_port
        self.gossip_port = gossip_port
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)

# Multiple version

In [4]:
peers = {}
#streetno["1"] = "Sachin Tendulkar" 

In [97]:
import os
import json
import subprocess
import pandas as pd

def getPeerAddress(peer):
    addr='peer0.kubernetes.research.dev.seeburger.de'
    return str(addr + ':' + peer.listen_port)

def getPeerAddress0(peer):
    addr='kubernetes.research.dev.seeburger.de'
    return str(addr + ':' + peer.listen_port)

def initpeer(peername):
    pod_name = getPod(peername)
    listen_port = getPort(peername=peername, name ='peer-listen')
    gossip_port = getPort(peername=peername, name ='peer-gossip')
    return kubernetespeer(name=peername,pod_name=pod_name,listen_port=listen_port,gossip_port=gossip_port)
   
def addPeer(peername):  
    if peername in peers.keys():
        global peer
        peer = peers[peername]
        pod_name_text.value = peers[peername].pod_name
        peer_addr_text.value = getPeerAddress(peer)
        return
    os.environ['PEER_NAME'] = peername
    !cd /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/ && /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/configure-deployment.sh -n $PEER_NAME
    !/home/jovyan/work/usr/bin/kubectl apply -f /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/$PEER_NAME/k8s-peer-service.yaml
    print('done')
    setPeerPorts(peername)
    res = createConfig(peername=peername,peer_listen_port=os.environ['PEER_LISTEN_PORT'],peer_gossip_port=os.environ['GOSSIP_PORT'])
    print(res)
    !/home/jovyan/work/usr/bin/kubectl apply -f /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/$PEER_NAME/k8s-peer.yaml
    peer = initpeer(peername)
    peers[peername] = peer
    
def handle_submit(text):
    global peername
    peername = text.value
    addPeer(peername)
    
layout2 = {'width': '600px'}    
peertext      = addVisText(key='organization',layout=layout2)
pod_name_text = addVisText(key='pod name',disabled=True,layout=layout2)
peer_addr_text = addVisText(key='address',disabled=True,layout=layout2)
#pod_name_text.disabled=True

peertext.on_submit(handle_submit)      

Text(value='', description='organization', layout=Layout(width='600px'), style=DescriptionStyle(description_wi…

Text(value='', description='pod name', disabled=True, layout=Layout(width='600px'), style=DescriptionStyle(des…

Text(value='', description='address', disabled=True, layout=Layout(width='600px'), style=DescriptionStyle(desc…

In [ ]:
#deletePeer('tanja99')
for x in range(0,10):
    peerneme = 'schalke' + str(x)
    addPeer(peerneme)
    #deletePeer(peerneme)

In [7]:
#peer=initpeer('sally8')
#peer=peers['sally8']
peer.name,peer.pod_name, peer.listen_port
#'sally28' in peers.keys()

('schalke0', 'schalke0-6f6c7f585b-h82x2', '32478')

Step2: Sign by majority of old peers

In [26]:
# download current orgs
downloadToLocal(local_path='/tmp', local_name='org_msp.json' ,remote_path='upload')

with open('/tmp/org_msp.json', 'r') as fp:
    data = json.load(fp)

print(data)

for name in data:
    if name[:-3] in peers:
        peer = peers[name[:-3]]
        addBashEnvToPeer(peer)
        sign(peer.pod_name)

['Org1MSP', 'Org2MSP', 'schalke0MSP', 'schalke1MSP', 'schalke2MSP']


In [60]:
local_orgs=[]
for name in data:
    if name[:-3] in peers:
        local_orgs.append(name[:-3])

In [61]:
local_orgs

['schalke0', 'schalke1', 'schalke2']

Step 3

In [28]:
peer.pod_name

'schalke3-7dd96d8dc4-2d4pv'

In [29]:
#pod_name   = peers['sally5'].pod_name
orderer_ip = '10.15.128.97'   
orderer_hostname = 'orderer.example.com'
channel_name = 'mychannel'  
joinHyperledger(peer.pod_name,orderer_ip,orderer_hostname,channel_name)

b'fetch http://dl-cdn.alpinelinux.org/alpine/v3.12/main/x86_64/APKINDEX.tar.gz\nfetch http://dl-cdn.alpinelinux.org/alpine/v3.12/community/x86_64/APKINDEX.tar.gz\n(1/1) Installing curl (7.69.1-r3)\nExecuting busybox-1.31.1-r16.trigger\nOK: 29 MiB in 28 packages\n(1/17) Installing fstrm (0.6.0-r1)\n(2/17) Installing libgcc (9.3.0-r2)\n(3/17) Installing krb5-conf (1.0-r2)\n(4/17) Installing libcom_err (1.45.6-r0)\n(5/17) Installing keyutils-libs (1.6.1-r1)\n(6/17) Installing libverto (0.3.1-r1)\n(7/17) Installing krb5-libs (1.18.3-r0)\n(8/17) Installing json-c (0.14-r1)\n(9/17) Installing libstdc++ (9.3.0-r2)\n(10/17) Installing libprotobuf (3.12.2-r0)\n(11/17) Installing libprotoc (3.12.2-r0)\n(12/17) Installing protobuf-c (1.3.3-r1)\n(13/17) Installing libuv (1.38.1-r0)\n(14/17) Installing xz-libs (5.2.5-r0)\n(15/17) Installing libxml2 (2.9.10-r5)\n(16/17) Installing bind-libs (9.16.6-r0)\n(17/17) Installing bind-tools (9.16.6-r0)\nExecuting busybox-1.31.1-r16.trigger\nOK: 42 MiB in 45

B

# Enroll users

In [ ]:
!pwd


# Install and aprove chaincode

In [90]:
for org in local_orgs:
    print(org)
    addBashEnvToPeer(peers[org])
    
addBashEnvToPeer(peers['schalke3'])

schalke0
schalke1
schalke2


In [86]:
for name in data:
    if name[:-3] in peers:
        peer = peers[name[:-3]]
        installAndAproveChaincode(peer)

In [84]:
#installAndAproveChaincode(peers['schalke3'])
installAndAproveChaincode(peers['schalke0'])

In [67]:
downloadCrtsToPeer(local_orgs)

In [ ]:
peer lifecycle chaincode commit -o orderer.example.com:7050 --ordererTLSHostnameOverride orderer.example.com 
--tls --cafile $CERT_BASE_PATH/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem --channelID mychannel --name basic --peerAddresses peer0.org1.example.com:7051 --tlsRootCertFiles $CERT_BASE_PATH/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt --peerAddresses peer0.org2.example.com:9051 --tlsRootCertFiles ${CERT_BASE_PATH}/organizations/peerOrganizations/org2.example.com/peers/peer0.org2.example.com/tls/ca.crt --version 1.0 --sequence 1


In [74]:
# download e.g. all crts of needed peers to local dir (e.g. for invoke,commit)
def getCrtName(org='', domain='kubernetes.research.dev.seeburger.de'):
    return str(org + '-' + domain + '-ca.crt')
    
def downloadToLocalCmdList(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', remote_name='' ,remote_path='upload'):  
    return ['curl', '--user',user + ':' + pwd, 'http://' + host + '/' + remote_path + '/' + remote_name, '-o', local_path + '/' + remote_name]

def toCmd(strlist):
    return ''.join(str(e + ' ') for e in strlist)

def downloadCrtsToPeer(orgs, execute_org='schalke3'):
    for org in orgs:
        executePeerCmd(peers[execute_org].pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name=getCrtName(org))))     
   
#addBashEnvToPeer(peers['schalke3'])
downloadCrtsToPeer(local_orgs)
downloadCrtsToPeer(['schalke3'], execute_org='schalke3')

In [2]:
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
                    
                    
#downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

In [75]:
def commit():
    try:    
        #crtnew = '\" + getCrt(org='klaus') + '\"
        getCrt(org='klaus')
        output = subprocess.check_output(['peer', 'lifecycle','chaincode','commit','-o',addr0,'--ordererTLSHostnameOverride',orderer,
                                          '--tls','--cafile',ordererpem,'--channelID','mychannel','--name','basic',
                                          '--peerAddresses',addr1, '--tlsRootCertFiles', crt1, 
                                          '--peerAddresses', addr2, '--tlsRootCertFiles', crt2, 
                                          '--peerAddresses', 'peer0.scray.org', '--tlsRootCertFiles', '/tmp/new.crt',
                                          #'--peerAddresses', 'kubernetes.research.dev.seeburger.de:32190', '--tlsRootCertFiles', '/tmp/new.crt',
                                          '--version', '1.0', '--sequence', '1','--tlsRootCertFiles'])
        return output.decode('ascii')
    except Exception as e:
        return str(e)
    
def getCrt(org='',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt'])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)    


def getCrtToLocalCmd(org='',topath='/tmp',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    crtname = getCrtName(org, domain)
    return str('curl --user ' + user + ':' + pwd + \
            ' http://' + host + '/ca/' + channel + '/' + crtname +  ' -o ' +  ' /tmp/' + crtname)
        
        
def getCrtToLocal(org='',topath='/tmp',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/ca/' + channel + '/' + getCrtName(org, domain), '-o', '/tmp/' + getCrtName(org, domain)])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)       

commit chaincode

In [98]:
addr0 = 'orderer.example.com:7050'
orderer = 'orderer.example.com'
ordererpem = '/tmp/tlsca.example.com-cert.pem'
cmd1 = 'peer lifecycle chaincode commit -o ' + addr0 + ' --ordererTLSHostnameOverride ' + orderer + ' --tls --cafile ' + ordererpem + ' --channelID mychannel --name basic'
cmd2 = ' --version 1.0 --sequence 1'

def getPart(peer, org='kubernetes.research.dev.seeburger.de'):
    return getCrtToLocalCmd(org=peer.name)
    #return '--peerAddresses ' + org + ':' + peer.listen_port + ' --tlsRootCertFiles ' +  crt1

destpeer = peers['schalke3']    
for name in data:
    if name[:-3] in peers:
        peer = peers[name[:-3]]    
        cmd = getPart(peer)
        executePeerCmd(destpeer.pod_name,cmd)

addr='peer0.kubernetes.research.dev.seeburger.de'
crts=''
for name in data:
    if name[:-3] in peers:
        peer = peers[name[:-3]] 
        crts = crts + str(' --peerAddresses ' + getPeerAddress(peer)  + ' --tlsRootCertFiles ' +  '/tmp/' + getCrtName(org=peer.name))

In [99]:
executePeerCmd(destpeer.pod_name,cmd1 + crts + cmd2)

"Command '['/home/jovyan/work/usr/bin/kubectl', 'exec', '-it', 'schalke3-7dd96d8dc4-2d4pv', '-c', 'scray-peer-cli', '--', '/bin/bash', '-c', 'source /root/.bashrc && peer lifecycle chaincode commit -o orderer.example.com:7050 --ordererTLSHostnameOverride orderer.example.com --tls --cafile /tmp/tlsca.example.com-cert.pem --channelID mychannel --name basic --peerAddresses peer0.kubernetes.research.dev.seeburger.de:32478 --tlsRootCertFiles /tmp/schalke0-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30415 --tlsRootCertFiles /tmp/schalke1-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:32443 --tlsRootCertFiles /tmp/schalke2-kubernetes.research.dev.seeburger.de-ca.crt --version 1.0 --sequence 1']' returned non-zero exit status 1."

In [ ]:
crts

In [ ]:
peer lifecycle chaincode queryinstalled --peerAddresses peer0.kubernetes.research.dev.seeburger.de:32733 --tlsRootCertFiles /tmp/sally3-kubernetes.research.dev.seeburger.de-ca.crt

In [ ]:
pod_name   = peers['sally3'].pod_name
executePeerCmd(pod_name, "peer lifecycle chaincode queryinstalled --peerAddresses peer0.kubernetes.research.dev.seeburger.de:32733 --tlsRootCertFiles /tmp/sally3-kubernetes.research.dev.seeburger.de-ca.crt --output json")

In [ ]:
def readAsset(asset='asset1'):
    callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    try:    
        print(['peer', 'chaincode','query', '-C', 'mychannel', '-n', 'basic', '-c', callopt])
        #return json.loads(str(callProcess)[2:-3])
    except Exception as e:
        return str(e)
    
readAsset('asset1')   

In [ ]:
peer chaincode invoke -o orderer.example.com:7050 --tls true --cafile /tmp/tlsca.example.com-cert.pem  -C mychannel -n basic --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30084 --tlsRootCertFiles /tmp/sally1-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:31521 --tlsRootCertFiles /tmp/sally2-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:32733 --tlsRootCertFiles /tmp/sally3-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:31018 --tlsRootCertFiles /tmp/sally4-kubernetes.research.dev.seeburger.de-ca.crt -c '{"function":"CreateAsset","Args":["asset1","green","10", "Jin Soo","500"]}'
                        
peer chaincode query -C mychannel -n basic -c '{"function":"ReadAsset","Args":["asset1"]}'


In [ ]:

addBashEnvToPeer(peer.pod_name, peer.listen_port)

In [ ]:
print(pod_name)
executePeerCmd(pod_name,"hostname")

In [ ]:
executePeerCmd(pod_name,"peer chaincode query -C mychannel -n basic -c '{\"function\":\"GetAllAssets\",\"Args\":[]}'")

In [ ]:
executePeerCmd(pod_name,"peer chaincode query -C mychannel -n basic -c '{\"function\":\"ReadAsset\",\"Args\":[\"asset1\"]}'")

In [ ]:
executePeerCmd(pod_name,"peer chaincode query -C mychannel -n basic -c '{\"function\":\"ReadAsset\",\"Args\":[]}'")

In [ ]:
executePeerCmd(pod_name,"peer chaincode query -C mychannel -n basic -c '{\"function\":\"ReadAsset\",\"Args\":[\"asset1\"]}' --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30773 --tlsRootCertFiles /tmp/sally0-kubernetes.research.dev.seeburger.de-ca.crt")

In [ ]:
CreateAsset(ctx contractapi.TransactionContextInterface, id, color string, size int, owner string, appraisedValue int)

In [ ]:
executePeerCmd(pod_name, "peer chaincode query -C mychannel -n basic -c '{\"function\":\"ReadAsset\",\"Args\":[\"asset2\"]}'")

In [ ]:
executePeerCmd(pod_name,  "peer chaincode query -C mychannel -n basic -c '{\"function\":\"GetAllAssets\",\"Args\":[]}'")

In [ ]:
setPeerEnvironment(peername)

#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'lara2-5d8bd488d6-kll5n' -c lara2 -- /bin/bash pwd

In [ ]:
########## execute commands in container
#!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c maja3 -- /bin/sh -c "ls -l / && hostname"
#!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c scray-peer-cli -- /bin/bash -c "ls -l / && hostname"
#!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c scray-peer-cli -- /bin/bash -c "echo 'export ORDERER_CA=/tmp/tlsca.example.com-cert.pem' >> /root/.bashrc"

In [ ]:
#setPeerEnvironment('tanja2')
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  /bin/sh export ORDERER_CA="/tmp/tlsca.example.com-cert.pem"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  ORDERER_HOSTNAME="orderer.example.com"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  CHANNEL_NAME="mychannel"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  CORE_PEER_ADDRESS="peer0.kubernetes.research.dev.seeburger.de:32222"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  CORE_PEER_MSPCONFIGPATH="/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp"


!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- env | fgrep ORDERER_HOSTNAME

In [ ]:
peername='maja2'
setPeerEnvironment(peername)
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- env | fgrep ORDERER
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- which export
print(getPod(peername),getPort(peername=peername, name='peer-listen'))

In [ ]:
export ORDERER_CA=/tmp/tlsca.example.com-cert.pem
export ORDERER_HOSTNAME="orderer.example.com"
export CHANNEL_NAME="mychannel"
export CORE_PEER_ADDRESS="peer0.kubernetes.research.dev.seeburger.de:32190"
export CORE_PEER_MSPCONFIGPATH="/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp"
export SHARED_FS_HOST=10.15.136.41:30080
export SHARED_FS_USER=scray
export SHARED_FS_PW=scray

curl --user $SHARED_FS_USER:$SHARED_FS_PW http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem > /tmp/tlsca.example.com-cert.pem
curl --user scray:scray http://10.15.136.41:30080/upload/org3_update_in_envelope.pb -o ./org3_update_in_envelope.pb
peer channel signconfigtx -f org3_update_in_envelope.pb
curl --user scray:scray -T ./org3_update_in_envelope.pb http://10.15.136.41:30080/upload/org3_update_in_envelope.pb
  

In [ ]:
!ls -l ./*.pb

#sign update
#createEnvironmentForPeer(localmspid=mspid2, tlsrootcert=crt2, mspconfigpath=msp2, peeraddress=addr2)
#setPeerEnvironment('carla1')
setPeerEnvironment('tanja2')
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- curl --user scray:scray http://10.15.136.41:30080/upload/org3_update_in_envelope.pb -o ./org3_update_in_envelope.pb
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- peer channel signconfigtx -f org3_update_in_envelope.pb
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- curl --user scray:scray -T ./org3_update_in_envelope.pb http://10.15.136.41:30080/upload/org3_update_in_envelope.pb
  

# Install and aprove chaincode

In [ ]:
setPeerEnvironment('tanja2')
installAndAproveChaincode()

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- lifecycle chaincode querycommitted --channelID mychannel --name basic --output json

In [ ]:
#setPeerEnvironment('klaus')
!echo  $POD_NAME $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

# Host where the chaincode is running
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

In [ ]:
#!ping orderer.example.com 7050
!more /etc/hosts

# ------------------------------------------------------------------------

# Shell only

In [ ]:
#!echo $GOSSIP_PORT, $PEER_NAME
getPort(peername=peername, name='peer-listen')

In [ ]:
GOSSIP_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-gossip')].nodePort}")
PEER_LISTEN_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
PEER_CHAINCODE_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-chaincode')].nodePort}")


In [ ]:
!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-gossip')].nodePort}")
!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-chaincode')].nodePort}")

In [ ]:
os.environ['GOSSIP_PORT'] = "32068"  
os.environ['PEER_LISTEN_PORT'] = "30251"
os.environ['PEER_CHAINCODE_PORT'] = "30052"
!echo $GOSSIP_PORT
#!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-gossip')].nodePort}")

In [ ]:
!/home/jovyan/work/usr/bin/kubectl create configmap hl-fabric-peer-$PEER_NAME \
 --from-literal=hostname=kubernetes.research.dev.seeburger.de \
 --from-literal=org_name=$PEER_NAME \
 --from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:$PEER_LISTEN_PORT \
 --from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:$GOSSIP_PORT \
 --from-literal=CORE_PEER_LOCALMSPID=${PEER_NAME}MSP

In [ ]:
!/home/jovyan/work/usr/bin/kubectl apply -f /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/$PEER_NAME/k8s-peer.yaml

In [ ]:
POD_NAME=$(/home/jovyan/work/usr/bin/kubectl  get pod -l app=peer0-org1-scray-org -o jsonpath="{.items[0].metadata.name}")
kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME

In [ ]:
!echo $(/home/jovyan/work/usr/bin/kubectl  get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")


In [ ]:
os.environ['POD_NAME'] = "tom-6647c77bbb-hjlkc"
os.environ['ORDERER_IP'] = '10.15.128.97'
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
os.environ['CHANNEL_NAME'] = 'mychannel'

!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME

# Delete old peer

In [ ]:
!/home/jovyan/work/usr/bin/kubectl delete configmap hl-fabric-peer-klaus
!/home/jovyan/work/usr/bin/kubectl delete -f /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/klaus/k8s-peer.yaml
!/home/jovyan/work/usr/bin/kubectl delete service klaus

In [ ]:
#deletePeer('carla1')
deletePeer('ff')

In [ ]:
!/home/jovyan/work/usr/bin/kubectl get configmap 
!/home/jovyan/work/usr/bin/kubectl get deployments 
#!/home/jovyan/work/usr/bin/kubectl get svc -o json
!/home/jovyan/work/usr/bin/kubectl get svc

# Create new peer and 'apply'

In [ ]:
!/home/jovyan/work/usr/bin/kubectl create configmap hl-fabric-peer \
 --from-literal=hostname=kubernetes.research.dev.seeburger.de \
 --from-literal=org_name=OrgSally \ 
 --from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:30003 \
 --from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:30001 \
 --from-literal=CORE_PEER_LOCALMSPID=OrgSallyMSP

!/home/jovyan/work/usr/bin/kubectl apply -f https://raw.githubusercontent.com/scray/scray/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/k8s-peer.yaml        

1. Der neue Peer wurde erzeugt. All relevant infos and cryptos for the application is uploaded to external sharepoint
2. Now the application needs to be signed by other peers

# Now the peer can join the network

In [ ]:
import os
import json
import subprocess
import pandas as pd

!/home/jovyan/work/usr/bin/kubectl get pods
os.environ['POD_NAME'] = 'peer0-org1-scray-org-5c4d69f556-snz5c'

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl get pods | fgrep peer0
#!POD_NAME=$(/home/jovyan/work/usr/bin/kubectl get pod -l app=peer0-org1-scray-org | tail -n 1 | cut -d' ' -f1)
os.environ['ORDERER_IP'] = '10.15.128.97'
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
os.environ['CHANNEL_NAME'] = 'mychannel'

!echo $POD_NAME
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP $ORDERER_HOSTNAME $CHANNEL_NAME

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty peer0-org1-scray-org-84ddc5757f-glbgn -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

# Install and aprove chaincode

In [ ]:
# Host where the chaincode is running
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

# TMP

In [ ]:
print(getPod(peername),getPort(peername=peername, name='peer-listen'))

In [ ]:
export ORDERER_CA="/tmp/tlsca.example.com-cert.pem"
export ORDERER_HOSTNAME="orderer.example.com"
export CHANNEL_NAME="mychannel"
export CORE_PEER_ADDRESS="peer0.kubernetes.research.dev.seeburger.de:32190"
export CORE_PEER_MSPCONFIGPATH="/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp"

peer lifecycle chaincode querycommitted --channelID mychannel --name basic --output json
   

In [ ]:
def readAsset(asset='asset1'):
    callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    try:    
        callProcess  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- peer', 'chaincode','query', '-C', 'mychannel', '-n', 'basic', '-c', callopt])
        return json.loads(str(callProcess)[2:-3])
    except Exception as e:
        return str(e)
    
readAsset('asset1')    

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- peer lifecycle chaincode querycommitted --channelID mychannel --name basic --output json

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- peer

In [ ]:
#pip install PyYAML
import yaml